In [1]:
import boto3
import sagemaker
import os
from sagemaker import get_execution_role

region = boto3.session.Session().region_name
role = get_execution_role()

# 如果在自建EC2无法获取role， 可以手动复制 role
#  role = arn:aws-cn:iam::账户id:role/service-role/AmazonSageMaker-ExecutionRole-20200430T123312

---------------------------------
## 第一步 创建ECR 

In [2]:
# Run this cell only onece to create the repository in ECR

account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'ocr-inference-container'
tag = ':latest'
uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'
inference_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)
print(inference_repository_uri)
ecr = '{}.dkr.ecr.{}.{}'.format(account_id, region, uri_suffix)



690704700794.dkr.ecr.cn-northwest-1.amazonaws.com.cn/ocr-inference-container:latest


*  第一次需要创建 ECR仓库

In [ ]:
!aws ecr create-repository --repository-name $ecr_repository

------------------------------------
## 第二步 构建镜像

### 镜像使用说明

* 本次使用的镜像 
727897471807.dkr.ecr.cn-northwest-1.amazonaws.com.cn/pytorch-inference:1.5.0-gpu-py36-cu101-ubuntu16.04

*  [可使用的Docker镜像列表](https://aws.amazon.com/cn/releasenotes/available-deep-learning-containers-images/) 

*  不需要修改账号Id   
727897471807 是官方id, 下载官方镜像使用


### Build and push

In [3]:
!aws ecr get-login-password --region cn-northwest-1 | docker login --username AWS --password-stdin 727897471807.dkr.ecr.cn-northwest-1.amazonaws.com.cn

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [4]:
%%time
!aws ecr get-login-password --region $region | docker login --username AWS --password-stdin $ecr

# Create ECR repository and push docker image
!docker rmi -f $inference_repository_uri
!docker build -t $ecr_repository ./

!docker tag {ecr_repository + tag} $inference_repository_uri
!docker push $inference_repository_uri

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Error: No such image: 690704700794.dkr.ecr.cn-northwest-1.amazonaws.com.cn/ocr-inference-container:latest
Sending build context to Docker daemon  514.6kB
Step 1/15 : FROM 727897471807.dkr.ecr.cn-northwest-1.amazonaws.com.cn/pytorch-inference:1.5.0-gpu-py36-cu101-ubuntu16.04
 ---> fc4de87c9036
Step 2/15 : RUN apt-get install -y --no-install-recommends nginx     && rm -rf /var/lib/apt/lists/*
 ---> Running in 9a5cb6f308b8
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libgd3 libgeoip1 libjbig0 libtiff5 libvpx3 libxpm4 libxslt1.1 nginx-common
  nginx-core
Suggested packages:
  libgd-tools geoip-bin fcgiwrap nginx-doc ssl-cert
Recommended packages:
  geoip-databas

将现有镜像下载到本地，并推送到自己的ECR库中

------------------------------------
## 第三步 准备训练好模型文件



**将如下model_uri改为在training阶段得到的模型在S3中的path，形式为s3://YOUR_BUCKET/sagemaker-ocr-chinese/output/-x-x-x-x-x-x-x/output/model.tar.gz**， 
可以在console找到该训练任务，在该训练任务的描述页面中，找到“S3 模型构件”，复制即可。

In [5]:
image = inference_repository_uri

model_uri = 's3://dikers-public/ocr-model/model.tar.gz'           #已经训练好的模型

#FIXME: 根据train 生成的模型， 替换下面的模型地址
#model_uri = 's3://YOUR_BUCKET/sagemaker-ocr-chinese/output/-x-x-x-x-x-x-x/output/model.tar.gz'

------------------------------------
##  第四步 准备测试文件

**即发送推理请求前，先将待推理的图片文件上传到S3**

推理请求的结构是发送一个json结构体，json结构体里面描述：

bucket: 存储桶

image_uri:待推理数据的uri，不含桶名

**s3://dikers-public/sagemaker-ocr-chinese/test-images/test001.jpg**   测试图片， 也可以替换自己S3里面的图片

```
{"bucket": "dikers-public", "image_uri": "sagemaker-ocr-chinese/test-images/test001.jpg"}
```



<img style="width:400px;height:250px" src="https://dikers-public.s3.cn-northwest-1.amazonaws.com.cn/sagemaker-ocr-chinese/test-images/test001.jpg" />

In [6]:
import json
bucket = 'dikers-public'
image_uri = 'sagemaker-ocr-chinese/test-images/test001.jpg'

test_data = {
    'bucket' : bucket,
    'image_uri' : image_uri
}
payload = json.dumps(test_data)
print(payload)

{"bucket": "dikers-public", "image_uri": "sagemaker-ocr-chinese/test-images/test001.jpg"}


------------------------------------
## 第五步 创建Endpoint 

### Method 1: Using sagemaker SDK

In [7]:
# 创建 model
# Below could be modified as you want

initial_instance_count = 1
instance_type = 'ml.m5.large'
endpoint_name= 'ocr-endpoint'

from sagemaker.model import Model
image = inference_repository_uri
model = Model(
            model_data=model_uri, 
            role=role,
            env={'test':'abc'},
            image=image)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


In [8]:
%%time

model.deploy(
    initial_instance_count=initial_instance_count,
    instance_type=instance_type,
    endpoint_name=endpoint_name)

-----------------!CPU times: user 320 ms, sys: 32.3 ms, total: 352 ms
Wall time: 8min 32s


------------------------------------
## 第六步  调用Endpoint

###  Method 1 客户端调用

In [9]:
# 创建推理用的 predictor

new_predictor = sagemaker.predictor.RealTimePredictor(
    endpoint=endpoint_name,
    content_type='application/json')

* 会返回AWS Textract 格式的json数据

In [11]:
%%time
# 推理请求代码

new_sm_response = new_predictor.predict(payload)
result = json.loads(new_sm_response.decode())
print(type(result))
print(result)


<class 'dict'>
{'DocumentMetadata': {'Pages': 1}, 'JobStatus': 'SUCCEEDED', 'Blocks': [{'BlockType': 'PAGE', 'Geometry': {'BoundingBox': {'Width': 1.0, 'Height': 1.0, 'Left': 0.0, 'Top': 0.0}, 'Polygon': [{'X': 0.0, 'Y': 0.0}, {'X': 1.0, 'Y': 0.0}, {'X': 1.0, 'Y': 1.0}, {'X': 0.0, 'Y': 1.0}]}, 'Id': 'e0eb46e8-8c56-4889-a06a-851e533232ff', 'Relationships': [{'Type': 'CHILD', 'Ids': ['70401d8d-815f-4d0b-8165-a2d23fdcc828', 'ab012d53-b849-4176-a5b0-e6370601c491', 'ebd3045c-7451-41a8-a6c9-7ac20ca02a33', '43a9c80e-6cf4-4361-a1e2-5ba91b409dd7', '68a5bdf7-3522-4b25-a8b8-7a1dd6f8e1f0', '8591c663-fab2-4a5f-b8ff-2bfbf7db5e44']}], 'Page': 1}, {'BlockType': 'WORD', 'Confidence': 0.99, 'Text': 'Hello', 'Geometry': {'BoundingBox': {'Width': 0.12311015118790497, 'Height': 44.0, 'Left': 0.12311015118790497, 'Top': 0.1079136690647482}, 'Polygon': [{'X': 0.12311015118790497, 'Y': 0.1079136690647482}, {'X': 0.24622030237580994, 'Y': 0.1079136690647482}, {'X': 0.24622030237580994, 'Y': 0.18705035971223022

In [12]:
with open('test.json', 'w') as f:
    json.dump(result, f)
    f.close()

##  完成测试
-------------------------------------------------

## Boto3 SDK  部署和测试方式

### Method 2: Using boto3 SDK

In [ ]:
# Below could be modified as you want

model_name = 'ocr-demo'
endpoint_config_name='ocr-endpoint-config'
variant_name= 'ocr-endpoint-vn'
initial_instance_count = 1
instance_type = 'ml.m5.large'
endpoint_name= 'endpoint_config_name'

In [ ]:
import boto3

sm_client = boto3.client('sagemaker')

# create model object

spl_model_demo = sm_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': image,
        'Mode': 'SingleModel',
        'ModelDataUrl': model_uri,
    },
    ExecutionRoleArn= role, 
    EnableNetworkIsolation=False
)

In [ ]:
# create endpoint config

spl_endpoint_config = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': variant_name,
            'ModelName': model_name,
            'InitialInstanceCount': initial_instance_count,
            'InstanceType': instance_type
        },
    ]
)

In [ ]:
# create endpoint

response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

待上一步创建完成后再进行下面的发送推理请求。上面创建endpoint的时间大概10分钟左右，可以在console查看状态，inservice即可使用了。

###  Method 2 客户端调用

In [ ]:
import boto3
import json
import time

region_name='cn-northwest-1'
profile_name='default'

session = boto3.session.Session(region_name=region_name, profile_name=profile_name)
client = session.client('sagemaker-runtime')

start_time = time.time()
spl_response=client.invoke_endpoint(EndpointName=endpoint_name,
        Body=payload,
        ContentType='application/json')
end_time = time.time()

print('time cost %s s' %(end_time - start_time))
print(json.loads(spl_response['Body'].read().decode()))